# Welcome to my Kaggle Survey Kernel 

I'm focused on Computer Vision tasks, but when I saw the Survey data, I decided to take it an attention. 

I was thinking about the difference in Age Range of Kagglers and I will follow this idea.

The questions that I will go further will be:
> - What's the Age Range of the Kagglers?
- We have the same Age Range pattern in all countries?
- What is the distribution of Genders by different Age Ranges?
- We can see difference in time to respond the survey by different Age Ranges?
- The salary of diferent Age Ranges are the same ?
- What's the formal education differences? <br>

And many more interesting questions... 

# NOTE: This kernel is not finished; 
If you find the kernel useful, dont hesitate in upvote the kernel, please =) 

# Importing Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_q = pd.read_csv('/kaggle/input/kaggle-survey-2019/questions_only.csv', low_memory=False)
df_mcr = pd.read_csv('/kaggle/input/kaggle-survey-2019/multiple_choice_responses.csv', header=[1], low_memory=False)
df_otr = pd.read_csv('/kaggle/input/kaggle-survey-2019/other_text_responses.csv', low_memory=False)
df_sc = pd.read_csv('/kaggle/input/kaggle-survey-2019/survey_schema.csv', low_memory=False)

# Understanding the Distribution by Age
- Whats the age distribution of Kagglers?

In [ ]:
kaggler_ages = df_mcr['What is your age (# years)?'].value_counts(dropna=False).sort_index()

total = len(df_mcr)

plt.figure(figsize=(10,5))

g = sns.barplot(x=kaggler_ages.index, y=kaggler_ages.values, color='blue')
g.set_title("Kagglers Age Distribution", fontsize=22)
g.set_xlabel("Age Range", fontsize=19)
g.set_ylabel("Total Count", fontsize=19)

sizes=[]
for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x()+p.get_width()/2.,
            height + 35,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=11) 
g.set_ylim(0, max(sizes) * 1.1)


plt.show()

Cool! Now we can have some sense of the Age distribution of Kagglers. <br>

>- 25-29 is the most common age range
>- 22-24 is the second most common age range
>- 30-34 is the third most commmon age range 

I think that should be interesting to find for more interesting patterns and differences by age.<br>
In the below sections I intend to segment this Kagglers in 3 different groups. But before it, lets see the difference to answer the complete form by the ages. 

## I will create a new column called "Age_range" and merge some groups in a unique.
- Take a look to understand the new arrangement that will give us a clearly vision about the etary factors;<br>

I did the segmentation based on my perception, separting by generations. 

In [ ]:
df_mcr.loc[df_mcr['What is your age (# years)?'].isin(['18-21', '22-24']), 'age_range'] = '1 - Young'
df_mcr.loc[df_mcr['What is your age (# years)?'].isin(['25-29', '30-34']), 'age_range'] = '2 - Young Adult'
df_mcr.loc[df_mcr['What is your age (# years)?'].isin(['35-39', '40-44']), 'age_range'] = '3 - Adult'
df_mcr.loc[df_mcr['What is your age (# years)?'].isin(['45-49', '50-54', '55-59', '60-69', '70+']), 'age_range'] = '4 - Sr. Adult'

age_r_ord = df_mcr['age_range'].value_counts(dropna=False).sort_index()

# Survey Duration by different Age Ranges
- Do we have some difference in the duration of the survey for different age ranges? 

In [ ]:
df_mcr['Duration (in seconds)'] = round(df_mcr['Duration (in seconds)'] / 60,2)

### How many people take more than 60 min to answer the form? 

In [ ]:
min_thresh = 60
print(f"Lenght of Answers < or equal {min_thresh} minutes:  # \
            {len(df_mcr['Duration (in seconds)'][df_mcr['Duration (in seconds)'] <= min_thresh])}")
print(f"Lenght of Answers > {min_thresh} minutes:           # \
            {len(df_mcr['Duration (in seconds)'][df_mcr['Duration (in seconds)'] > min_thresh])}")

print("\nPercentiles of Duration: ")
print(df_mcr['Duration (in seconds)'].quantile([.05,.25,.5,.75,.95]))

Cool! We can see that we have some extreme values in time to fill the survey, so to a better visualization and understanding of the distribution, I will set a limit to the plots below. 



## Ploting Duration (in seconds) by Age Range

In [ ]:
plt.figure(figsize = (16,5))

plt.subplot(1,2,1)
g = sns.distplot(df_mcr[df_mcr['Duration (in seconds)'] <= 200]['Duration (in seconds)'])
g.set_xlabel('Duration minutes distribution', fontsize=15)
g.set_title("Histogram of Duration", fontsize=19)

plt.subplot(1,2,2)
plt.scatter(range(df_mcr.shape[0]), np.sort(df_mcr['Duration (in seconds)'].values))
plt.xlabel('Range of dataset', fontsize=15)
plt.title("Distribution of Duration ", fontsize=19)

plt.show()

# Duration by Age Range

In [ ]:
plt.figure(figsize=(12,6))

g = sns.boxplot(x='What is your age (# years)?', y='Duration (in seconds)', 
                data = df_mcr[df_mcr['Duration (in seconds)'] <= 50], color='blue',
               order=kaggler_ages.index)
g.set_title("Duration(in scnds) Distribution by Different Age Ranges", fontsize=20)
g.set_xlabel("Age Range", fontsize=17)
g.set_ylabel("Duration(Minutes) distribution", fontsize=17)

plt.show()

Cool. Interestingly we can see that the yougest kaggler taked less time to fill all the survey. 

I think that it could be because of the youngest guys have less experiences to fill the form.... I will investigate it further later.  

# Gender by Age Range
- Let's see if the Gender ratio are different in different Age Ranges

In [ ]:
age = df_mcr.groupby(['age_range', 'What is your gender? - Selected Choice'])['age_range'].count().unstack('What is your gender? - Selected Choice').reset_index()

In [ ]:
plt.figure(figsize=(10,5))

g = sns.countplot(x='age_range',
                  hue='What is your gender? - Selected Choice', 
                  data = df_mcr,
                  #color='blue', 
                  order=age_r_ord.index
               )
g.set_title("Duration(in minutes) Distribution by Daifferent Age Ranges", fontsize=18)
g.set_xlabel("Age Range", fontsize=16)
g.set_ylabel("Total Count", fontsize=16)

plt.show()

Cool. 
- 82% of total Kagglers are Men. 
- 16% of total Kagglers are Women.
- ~2% of total Kagglers not defined the gender. 

The highest ratio of Female / Male is in the youngest groups. 

# Age Distribution by the top 15 countries

In [ ]:
df_mcr.loc[df_mcr["In which country do you currently reside?"] == 
                   'United Kingdom of Great Britain and Northern Ireland', "In which country do you currently reside?"] = "UK & N. Ireland"
df_mcr.loc[df_mcr["In which country do you currently reside?"] == 
                   'United States of America', "In which country do you currently reside?"] = "USA"

plt.figure(figsize=(16,14))
plt.subplot(2,1,1)
g = sns.countplot(x='In which country do you currently reside?',
                  #hue='age_range', 
                  data = df_mcr, color='blue',
                  #hue_order=age_r_ord.index,
                  #color='blue', 
                  order=df_mcr['In which country do you currently reside?'].value_counts()[:15].index
               )
g.set_title("TOP 15 Countries with more Kagglers", fontsize=18)
g.set_xlabel("Country Names", fontsize=16)
g.set_ylabel("Count", fontsize=16)
g.set_xticklabels(g.get_xticklabels(),rotation=15)
sizes=[]
for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x()+p.get_width()/2.,
            height + 35,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=11) 
g.set_ylim(0, max(sizes) * 1.1)

plt.subplot(2,1,2)
g = sns.countplot(x='In which country do you currently reside?',
                  hue='age_range', 
                  data = df_mcr,
                  hue_order=age_r_ord.index,
                  #color='blue', 
                  order=df_mcr['In which country do you currently reside?'].value_counts()[:15].index
               )
g.set_title("AGE RANGE by TOP 15 Countries with more Kagglers", fontsize=18)
g.set_xlabel("Country Names", fontsize=16)
g.set_ylabel("Count", fontsize=16)
g.set_xticklabels(g.get_xticklabels(),rotation=15)

plt.subplots_adjust(hspace = 0.3, top = 0.8)

plt.show()

First Graph:
It's very interesting because now we have a very different pattern than the last year survey. India have almost 25% of total respondents. 

Second Graph:
It's very cool and insightful information !
In India the most Kagglers are youngest than other countries, with a very different pattern than other countries. 
If we take India and USA to compare, we can see that the ratio of Young / Young Adult are very different. <br>
Also if we consider the difference in Young / Senior Adults;

I think it's really very insightful to understand the future. 

In the top 15 countries, only India and China the young Kagglers are most common than y. Adults or Adults.... 

# Currently Yearly Compensation Distribution 

In [ ]:
salary_ord = ['$0-999', '1,000-1,999', '2,000-2,999', '3,000-3,999', '4,000-4,999','5,000-7,499', '7,500-9,999',
             '10,000-14,999', '15,000-19,999',  '20,000-24,999', '25,000-29,999',  '30,000-39,999', '40,000-49,999', 
             '50,000-59,999', '60,000-69,999', '70,000-79,999', '80,000-89,999', '90,000-99,999', '100,000-124,999',
             '125,000-149,999', '150,000-199,999', '200,000-249,999', '250,000-299,999', '300,000-500,000', '> $500,000']

df_mcr.rename(columns={'What is your gender? - Selected Choice':'Gender'}, inplace=True)

plt.figure(figsize=(16,15))
print(f"Total of Not informed values: {df_mcr['What is your current yearly compensation (approximate $USD)?'].isna().sum()}")

plt.subplot(2,1,1)
g = sns.countplot(x='What is your current yearly compensation (approximate $USD)?',
                  #hue='age_range', 
                  data = df_mcr, #hue='Gender',
                  #hue_order=['Male', 'Female'],
                  color='blue', 
                  order=salary_ord
               )
g.set_title("Yearly Compensation Range Distributions", fontsize=18)
g.set_xlabel("Yearly Compensation - USD", fontsize=16)
g.set_ylabel("Count", fontsize=16)
g.set_xticklabels(g.get_xticklabels(),rotation=45)

plt.subplot(2,1,2)
g = sns.countplot(x='What is your current yearly compensation (approximate $USD)?',
                  #hue='age_range', 
                  data = df_mcr, 
                  hue='age_range',
                  hue_order=age_r_ord.index,
                  #color='blue', 
                  order=salary_ord
               )
g.set_title("Yearly Compensation Range Distributions by Age Range", fontsize=18)
g.set_xlabel("Yearly Compensation - USD", fontsize=16)
g.set_ylabel("Count", fontsize=16)
g.set_xticklabels(g.get_xticklabels(),rotation=45)

plt.subplots_adjust(hspace = 0.5, top = 0.8)

plt.show()

Cool! We can see that youngest guys have yearly earning 

In [ ]:
age_r_ord

'Master’s degree', 'Bachelor’s degree', 'Doctoral degree',
       'Some college/university study without earning a bachelor’s degree',
       'Professional degree', 'I prefer not to answer',
       'No formal education past high school'

## Level of Formal Education

In [ ]:
df_mcr.loc[df_mcr['What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'] == \
           'Some college/university study without earning a bachelor’s degree', 
           'What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'] = 'Some college w/o Bs.C'

df_mcr.loc[df_mcr['What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'] == \
           'No formal education past high school', 
           'What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'] = 'No formal educ'

In [ ]:
plt.figure(figsize=(15,6))

educ_ord = ['I prefer not to answer', 'No formal educ', 'Some college wo Bs.C',
            'Professional degree', 'Bachelor’s degree', 'Master’s degree',
            'Doctoral degree']

g = sns.countplot(x='What is the highest level of formal education that you have attained or plan to attain within the next 2 years?',
                  hue='age_range', 
                  data = df_mcr, order=educ_ord,
                  hue_order=age_r_ord.index,
                  #color='blue', 
               )
g.set_title("Level of Formal Education attained or planned to obtain in next 2 years", fontsize=18)
g.set_xlabel("Level of Formal Education", fontsize=16)
g.set_ylabel("Count", fontsize=16)
g.set_xticklabels(g.get_xticklabels(),rotation=15)

plt.show()

Nice ! We can see some very interesting patterns. 


# Current Role Position

In [ ]:
df_mcr.loc[df_mcr['Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].isnull(), 
           'Select the title most similar to your current role (or most recent title if retired): - Selected Choice'] = "None"

# df_mcr['Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].value_counts(dropna=False)

In [ ]:
plt.figure(figsize=(16,6))

g = sns.countplot(x='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',
                  hue='age_range', 
                  data = df_mcr, #order=educ_ord,
                  hue_order=age_r_ord.index,
                  #color='blue', 
               )
g.set_title("Most Similar Title of the current role of Kagglers", fontsize=18)
g.set_xlabel("Current Title Position", fontsize=16)
g.set_ylabel("Count", fontsize=16)
g.set_xticklabels(g.get_xticklabels(),rotation=15)

plt.show()

Cool! We can note a big part of Kagglers don't work exactly with data science. What could explain why kaggle has so many new registrations but a few number of active members. 
As we could intuitively think, Yougest guys are in general students... It's the new generation. I'm sure that this new generation will be excellent scientists without some accents... They will speak the language of the "data science". 

In [ ]:
df_mcr['Does your current employer incorporate machine learning methods into their business?'].value_counts()

In [ ]:
plat_courses = df_mcr.loc[:,df_mcr.columns.str.startswith('On which platforms have you begun\
                                                or completed data science courses? (Select all that apply)')].columns

df_mcr.loc[:,df_mcr.columns.str.startswith('On which platforms have you begun or completed data science courses? (Select all that apply)')].isnull().mean().plot(kind='bar')



In [ ]:
plat_courses = df_mcr.loc[:,df_mcr.columns.str.startswith('On which platforms have you begun or completed data science courses? (Select all that apply)')].columns